In [20]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model, Input

In [7]:
link = 'D:/users/Marko/downloads/mirna/'

In [8]:
X_train = np.load(f'{link}/data/modmirbase_train_images.npz')['arr_0']/255
X_test = np.load(f'{link}/data/modmirbase_test_images.npz')['arr_0']/255
X_train_cat = np.load(f'{link}/data/modmirbase_train_images_cattt.npz')['arr_0']
X_test_cat = np.load(f'{link}/data/modmirbase_test_images_cattt.npz')['arr_0']

In [9]:
datagen_tr = ImageDataGenerator()
datagen_te = ImageDataGenerator()

train_gen = datagen_tr.flow(X_train, X_train_cat, batch_size=128)
test_gen = datagen_te.flow(X_test, X_test_cat, batch_size=128)

In [ ]:
class CVAE(tf.keras.Model):
  """Convolutional variational autoencoder."""

    def __init__(self, latent_dim):
        super(CVAE, self).__init__()
        self.latent_dim = latent_dim
        model = tf.keras.models.load_model('D:\\Users\\Marko\\Downloads\\deepmir-master\\models\\fine_tuned_cnn.h5')
        encoder_model = Model(inputs = model.input , outputs = model.layers[-4].output)
        for layer in encoder_model.layers:
            layer.trainable = False
            
        self.encoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(25, 100, 3)),
                encoder_model,
                # No activation
                tf.keras.layers.Dense(latent_dim + latent_dim),
            ]
        )

        self.decoder = tf.keras.Sequential(
            [
                tf.keras.layers.InputLayer(input_shape=(latent_dim,)),
                tf.keras.layers.Dense(units=7*7*32, activation=tf.nn.relu),
                tf.keras.layers.Reshape(target_shape=(7, 7, 32)),
                tf.keras.layers.Conv2DTranspose(
                    filters=64, kernel_size=3, strides=2, padding='same',
                    activation='relu'),
                tf.keras.layers.Conv2DTranspose(
                    filters=32, kernel_size=3, strides=2, padding='same',
                    activation='relu'),
                # No activation
                tf.keras.layers.Conv2DTranspose(
                    filters=1, kernel_size=3, strides=1, padding='same'),
            ]
        )

    @tf.function
    def sample(self, eps=None):
        if eps is None:
            eps = tf.random.normal(shape=(100, self.latent_dim))
        return self.decode(eps, apply_sigmoid=True)

    def encode(self, x):
        mean, logvar = tf.split(self.encoder(x), num_or_size_splits=2, axis=1)
        return mean, logvar

    def reparameterize(self, mean, logvar):
        eps = tf.random.normal(shape=mean.shape)
        return eps * tf.exp(logvar * .5) + mean

    def decode(self, z, apply_sigmoid=False):
        logits = self.decoder(z)
        if apply_sigmoid:
            probs = tf.sigmoid(logits)
            return probs
        return logits

In [13]:
#next(train_gen)[0].shape

(128, 25, 100, 3)

In [4]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [25]:
model2 = 

In [27]:
for l in model.layers:
    print(l.trainable)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
